# Create Model

이제 CNN 신경망을 만들 것이다.

In [7]:
#신경망 구성
inputs = layers.Input(shape=(PANNEL_SIZE, PANNEL_SIZE, 3))

net = layers.Conv2D(64, kernel_size=3, padding='same')(inputs)
# net = layers.Activation('relu')(net)
net = layers.LeakyReLU()(net)
net = layers.MaxPool2D(pool_size=2)(net) 

shortcut_1 = net

net = layers.Conv2D(128, kernel_size=3, padding='same')(net)
# net = layers.Activation('relu')(net)
net = layers.LeakyReLU()(net)
net = layers.MaxPool2D(pool_size=2)(net)

shortcut_2 = net

net = layers.Conv2D(256, kernel_size=3, padding='same')(net)
# net = layers.Activation('relu')(net)
net = layers.LeakyReLU()(net)
net = layers.MaxPool2D(pool_size=2)(net)

shortcut_3 = net

net = layers.Conv2D(256, kernel_size=1, padding='same')(net)
# net = layers.Activation('relu')(net)
net = layers.LeakyReLU()(net)
net = layers.MaxPool2D(pool_size=2)(net)

net = layers.UpSampling2D(size=2)(net)
net = layers.Conv2D(256, kernel_size=3, padding='same')(net)
net = layers.Activation('relu')(net)

net = layers.Add()([net, shortcut_3])

net = layers.UpSampling2D(size=2)(net)
net = layers.Conv2D(128, kernel_size=3, padding='same')(net)
net = layers.Activation('relu')(net)

net = layers.Add()([net, shortcut_2])

net = layers.UpSampling2D(size=2)(net)
net = layers.Conv2D(64, kernel_size=3, padding='same')(net)
net = layers.Activation('relu')(net)

net = layers.Add()([net, shortcut_1])

net = layers.UpSampling2D(size=2)(net)
net = layers.Conv2D(2, kernel_size=1, padding='same')(net)

net = layers.Reshape((-1, 2))(net)
net = layers.Activation('softmax')(net)

model = Model(inputs=inputs, outputs=net)

model.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizers.Adam(), 
    metrics=['acc'],
    sample_weight_mode='temporal'
)

model.summary()

NameError: name 'layers' is not defined

대략적인 과정은 이러하다.<br>
신경망은 DownSampling을 4번, UpSampling을 4번한 구조이다.<br>
MaxPool2D로 DownSampling을 하고 해당 층의 모델을 shortcut에 저장해둔다. 이 과정을 4번 반복하여 size를 축소시킨다. <br>
그 후에 UpSampling을 4번하고, 채널을 2개로 만들어준다. 그 후 두 채널을 더하여 1이 되는 값을 나오게 한다.

In [ ]:
#inputs = layers.Input(shape=(PANNEL_SIZE, PANNEL_SIZE, 3))

input 데이터는 224,224,3로, 224*224 사이즈의 컬러 이미지 3채널이다.

In [ ]:
# net = layers.Conv2D(64, kernel_size=3, padding='same')(inputs)
# # net = layers.Activation('relu')(net)
# net = layers.LeakyReLU()(net)
# net = layers.MaxPool2D(pool_size=2)(net) 

MaxPool2D를 이용해 DownSampling을 하는 과정이다.
input 데이터가 처음으로 들어가게 되는데, Conv2D를 통해서 데이터의 특징을 뽑아낸다. 합성곱을 사용하여 특징을 뽑아내는데, 필터를 통해 필터의 값과 합성곱을 하여 필요없는 값들을 작게 만들고, 중요한 값들은 값을 늘려 필요한 값은 커지고, 필요없는 값들은 작아지게 만든다.
* 64는 필터의 수
* kernel_size는 커널의 행과 열 (아래 사진은 kernel_size=5)
* padding은 합성곱을 하기 전에 주변을 특정값으로 채워넣는 것으로, 경계를 처리하기 위해 사용된다. padding에 따라 사이즈가 변경된다.

<img src = "https://blog.kakaocdn.net/dn/7xpIJ/btrxiXZwws7/QQckg2sKVSwApKcEDdChi0/img.gif" width = "300" height = "300">
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F9916C23F5BC97EEE31" width = "300" height = "300">


                Conv2D                       padding

신경망에서 노드에 들어오는 값들은 Activation function(활성화 함수)을 통과하여 다음 층에 들어갈 지 말지 결정한다. <br> 흔히 사용하는 ReLU를 사용하지 않고, LeakyReLU를 사용하는 이유는 ReLU는 x가 음수일 때 기울기가 0이 된다. 음수의 값들은 모델에 영향을 주지 않아 정보가 손실된다.<br>하지만 LeakyReLU는 x가 음수일 때 기울기를 0.01정도 주어 음수값도 미미하지만 영향을 주게되어 ReLU의 단점을 보완했다.

MaxPooling2D는 input size를 줄이는 과정으로, MaxPooling2D는 큰 값만 뽑아서 pool_size로 축소시킨다. 사진과 같이 2x2사이즈의 크기에서 가장 큰 값들만 뽑아서 2x2사이즈로 사이즈를 줄인다.<br>큰 값들만 추출하고 작은 값은 없앴기에 특징을 더 잘 뽑아낸다. 이로 인해 Overfitting(과적합)을 줄일 수 있다.

![alt text](./maxpooling2d.png)

이렇게 한 차례의 DownSampling을 하고 shortcut에 해당 모델을 저장하는데, 이는<span style="color: #2D3748; background-color:#fff5b1;">  **skip connection**  </span>을 사용하기 위해 저장하는 것이다.<br>
**skip connection**은 이를 사용하는 UpSampling과정에서 설명하겠다.

In [ ]:
# net = layers.UpSampling2D(size=2)(net)
# net = layers.Conv2D(256, kernel_size=3, padding='same')(net)
# net = layers.Activation('relu')(net)

DownSampling을 4번한 후에 UpSampling을 한다.<br>
UpSampling을 하는 이유는 DownSampling을 하면서 압축된 이미지를 원본 크기로 되돌리기 위함이다.<br>
UpSampling2D를 사용하면 사이즈를 2배씩 늘리는데, 해당 코드는 2X2사이즈로 4X4로 늘린다.

In [ ]:
# net = layers.Add()([net, shortcut_3])

shortcut을 UpSampling 모델을 만든 후에 넣는데, 그 이유는 skip connection을 사용하기 위해서이다.<br>
신경망 모델은 층이 깊을 수록 좋은 성능을 보여준다. 하지만 층이 너무 깊으면 오히려 성능이 저하되는데, 이를 해결하기 위해 skip connection을 사용한다.

사진과 같이 일반적인 구조에서는 입력 x를 받아 2개의 weighted layer를 거쳐 H(x)가 출력된다.<br>
skip connection을 적용한 구조 Residual 구조는 weighted layer를 거친 값 F(x)와, 입력 값 x를 더한다.<br>
더하는 것의 의미는 기존에 학습한 정보는 보존하고 새로운 학습 정보를 더하는 의미이다. 즉 해당 층에서는 추가적으로 학습해야 할 정보만 학습하게 만드는 것이다.

해당 코드에서는 인풋값 대신에 같은 크기의 UpSampling한 모델을 넣는다.

![skipconnect](https://gaussian37.github.io/assets/img/dl/concept/resnet/2.png "일반적인 모델과, skipconnect을 적용한 모델")

![alt text](./plain1.png)
![alt text](./skipconnection1.png)

                        plain model                                         skip connection model

시간상 epochs를 50으로 주었다.

직접 실행했을 때, 학습할 때마다 다르지만 대체적으로 사진과 같이 skip connection을 적용한 모델이 적용하지 않은 모델보다 정확도가 향상된다.

In [ ]:
# net = layers.Reshape((-1, 2))(net)
# net = layers.Activation('softmax')(net)

4번의 UpSampling 과정을 끝내면 원본 이미지의 사이즈로 돌아오게 된다.<br>
Reshape로 sample_weights에서 정의한 output형태로 모양을 변경해준다. (224*224,2)
softmax를 통해 두 채널을 합쳐서 1이 나오게한다.

In [ ]:
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer=optimizers.Adam(), 
#     metrics=['acc'],
#     sample_weight_mode='temporal'
# )

모델을 compile한다.

loss는 손실함수를 지정하는 것으로, categorical_crossentropy로 설정 (softmax에 주로 쓰임)<br>
optimizer는 Adam으로 설정<br>
metrics는 acc로, 학습과정에서 정확도를 수집한다.<br>
sample_weight_mode='temporal'은 손실 함수에 가중치를 적용하는데 사용된다.
